# Cell Inference Analisis Sentimen JKN Mobile

Cell ini digunakan untuk memprediksi sentimen dari suatu kalimat input. Model yang digunakan adalah Bi-LSTM yang telah dilatih sebelumnya menggunakan dataset sentimen.  

Cara penggunaan:
1. Masukkan kalimat yang ingin diprediksi pada input yang disediakan.
2. Model akan memproses kalimat dan menampilkan prediksi sentimen (positif, negatif, atau netral).

Catatan:
* Akurasi model bergantung pada kualitas dan kuantitas data latih yang digunakan.
* Model telah dioptimalkan untuk mendeteksi sentimen dalam konteks tertentu dan mungkin kurang akurat pada konteks yang berbeda.
* Anda dapat meningkatkan akurasi model dengan menambahkan data latih yang lebih beragam dan relevan.

Contoh penggunaan:

In [3]:
!pip install tensorflow scikit-learn

In [24]:
import re
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Download the model, tokenizer, and label encoder
!wget -q https://raw.githubusercontent.com/lailarizzah/analisis-sentimen-jkn-mobile/main/data%20cell%20inference/bilstm_model.keras -O bilstm_model.keras
!wget -q https://raw.githubusercontent.com/lailarizzah/analisis-sentimen-jkn-mobile/main/data%20cell%20inference/tokenizer.pkl -O tokenizer.pkl
!wget -q https://raw.githubusercontent.com/lailarizzah/analisis-sentimen-jkn-mobile/main/data%20cell%20inference/label_encoder.pkl -O label_encoder.pkl

model = tf.keras.models.load_model("bilstm_model.keras")

with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# --- Add custom words here, including strong negative words --- #
new_words = ['lemot', 'loading lama', 'sering macet', 'bagus', 'mantap', 'mudah', 'sulit', 'ribet', 'buruk',
            'bajingan', 'sampah', 'goblok', 'tolol', 'anjing', 'bangsat']  # tambahkan kata-kata kasar lainnya
for word in new_words:
    if word not in tokenizer.word_index:
        tokenizer.word_index[word] = len(tokenizer.word_index) + 1
# --- End of custom words --- #

with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www.\S+", "", text)
    # Modifikasi regex untuk mempertahankan tanda seru dan tanda tanya
    text = re.sub(r"[^a-zA-Z0-9\s!?]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Fungsi untuk memproses kalimat sebelum prediksi
def preprocess_sentence(sentence, tokenizer, max_len):
    cleaned = clean_text(sentence)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    return padded

# Dapatkan panjang maksimum input untuk model
# Instead of using model.input_shape, get maxlen from the training parameters or the tokenizer
# Assuming you used maxlen = 100 during training:
max_length = 100

# Input kalimat dari pengguna
kalimat_baru = input("Masukkan kalimat baru: ")

# Proses kalimat dan lakukan prediksi
X_baru = preprocess_sentence(kalimat_baru, tokenizer, max_length)
pred_prob = model.predict(X_baru)
pred_label = np.argmax(pred_prob)

# Tampilkan hasil prediksi
print("Sentimen kalimat tersebut adalah:", label_encoder.inverse_transform([pred_label])[0])

Masukkan kalimat baru: aplikasi jkn biasa aja sih
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
Sentimen kalimat tersebut adalah: neutral
